### Install optuna

In [ ]:
!pip install optuna==2.4.0

### import libraries

In [ ]:
import numpy as np
import optuna
from google.colab import drive

### load the data to google colab. We recommend the usage of google drive.



In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Define the parameters of the database: 1) file name and 2) study name. We are going to read the file containing the information about the trials performed when training the network on the magnetic fields.

In [ ]:
fdatabase  = 'sqlite:////content/drive/MyDrive/IllustrisTNG_o3_B_all_steps_500_500_o3.db'
study_name = 'wd_dr_hidden_lr_o3'

### Load the study; it contains the information of all trials. A trial is a the result of training a given neural network architecture with some value of the hyperparamters such as learning rate, weight decay...etc.

In [ ]:
study = optuna.load_study(study_name=study_name, storage=fdatabase)

### Get the number of trials in the study

In [ ]:
trials = len(study.trials)
print('Number of trials in the study: %d'%trials)

Number of trials in the study: 53


### read the losses and find the number of completed trials. The losses saved in the study are the minimum validation losses obtained during training.

In [ ]:
losses = np.zeros(trials)
completed_trials = 0
for i,t in enumerate(study.trials):
    losses[i] = t.value
    if t.value is not None:  completed_trials += 1
print('Number of completed trials: %d'%completed_trials)

Number of completed trials: 51


### sort the losses and print them. The smaller the loss the most accurate the model

In [ ]:
sorted_losses = np.sort(losses)
print(sorted_losses)

[-13.14501762 -13.06809807 -13.05419922 -13.02270794 -12.99445915
 -12.97795773 -12.97684002 -12.94909954 -12.93878365 -12.92854977
 -12.90500069 -12.9018383  -12.88184547 -12.88173676 -12.85134792
 -12.84923744 -12.81348705 -12.79772377 -12.7638855  -12.74578762
 -12.74035454 -12.72655106 -12.70239162 -12.6853056  -12.67906094
 -12.6699934  -12.61344337 -12.59365845 -12.45295143 -12.41748428
 -12.37825012 -12.35540295 -12.26841259 -12.2586937  -12.21970272
 -12.17620373 -12.02158356 -11.91000938 -11.9064312  -11.86744213
 -11.81512451 -11.63097763 -11.59908104 -11.5521574  -11.29914856
 -10.89695263 -10.78483486 -10.39631081 -10.27599716  -9.31357956
  -8.92316818          nan          nan]


### get the information about the best trial

In [ ]:
trial = study.best_trial
print("Best trial:  number {}".format(trial.number))
print("Loss:        %.5e"%trial.value)
print("Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:  number 39
Loss:        -1.31450e+01
Params: 
    dr: 0.3411427508944431
    hidden: 12
    lr: 0.00468987674696853
    wd: 3.535530203966222e-07


### print the information about a given trial

In [ ]:
trial_number = 20
trial = study.trials[trial_number]
print("Trial number:  number {}".format(trial.number))
print("Loss:          %.5e"%trial.value)
print("Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Trial number:  number 20
Loss:          -1.03963e+01
Params: 
    dr: 0.1452817436875268
    hidden: 7
    lr: 2.617531111544356e-05
    wd: 6.82909729255207e-08


### print information about the best 10 trials

In [ ]:
indexes = np.argsort(losses)
for trial_number in indexes[:10]:
  trial = study.trials[trial_number]
  print("\nTrial number:  number {}".format(trial.number))
  print("Loss:          %.5e"%trial.value)
  print("Params: ")
  for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


Trial number:  number 39
Loss:          -1.31450e+01
Params: 
    dr: 0.3411427508944431
    hidden: 12
    lr: 0.00468987674696853
    wd: 3.535530203966222e-07

Trial number:  number 46
Loss:          -1.30681e+01
Params: 
    dr: 0.490389911036187
    hidden: 12
    lr: 0.002683767595814597
    wd: 2.8244690099796116e-07

Trial number:  number 45
Loss:          -1.30542e+01
Params: 
    dr: 0.48741303970146455
    hidden: 12
    lr: 0.0032677443559415435
    wd: 1.0219686691224063e-06

Trial number:  number 42
Loss:          -1.30227e+01
Params: 
    dr: 0.33455450129588
    hidden: 12
    lr: 0.004713806068613913
    wd: 1.0356975744527914e-06

Trial number:  number 34
Loss:          -1.29945e+01
Params: 
    dr: 0.39565001257393007
    hidden: 12
    lr: 0.004769154258307381
    wd: 7.312666963332378e-07

Trial number:  number 40
Loss:          -1.29780e+01
Params: 
    dr: 0.3436651759572786
    hidden: 12
    lr: 0.0049477707762538405
    wd: 9.727048515507024e-07

Trial number

### With this information we can select the model we want, which typically will be the best trial (in this case trial 39). We need this information to find the file containing the weights of network, as well as to the value of ``hidden`` to test the model.
